In [1]:
from data.fwf_dataset import FwfDataset
from models.fgf import FGFeatNetwork
from utils.metrics import get_multilevel_metrics, print_metrics, combine_metrics_list, EarlyStopping, simple_metrics
from utils.general import generate_timestamp

from argparse import ArgumentParser
from tqdm import tqdm
from time import time
import numpy as np
import json
import os

import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from omegaconf import OmegaConf

# metrics computation alternatives
from sklearn.metrics import jaccard_score
from torchmetrics.segmentation import MeanIoU






In [2]:

args_test_grid_size = 0.8

args_exp_dir = './exp/2024-12-16_16-04-25_v006'
args_model_version = 'latest_model.pth'
args_return_full_probs = True
args_return_embedding = True



output_dir = os.path.join(args_exp_dir,'test')

# build config

cfg = OmegaConf.load(os.path.join(args_exp_dir,'config.yaml'))
with open(os.path.join(cfg.data.dataset_root, 'class_dict.json'),'r') as f:
    cfg = OmegaConf.merge(cfg, OmegaConf.create({'data':{'label_schema':json.load(f)}}))

if 'embedding_size' not in cfg.model.keys():
    cfg.model.embedding_size = 128
print(OmegaConf.to_yaml(cfg))

experiment:
  title_suffix: v005
  description: Is geometric information relevant (compare with v003 )
general:
  device: cuda
  batch_size: 1024
  weight_decay: 0.0005
  max_epochs: 25
  early_stopping_minDelta: 0.01
  num_workers: 4
data:
  dataset_root: ../../02_Datasets/FWF_Subsampled/0.01_simple/
  split: ./config/default_split.yaml
  preprocessing:
    _transformsTraining_:
      xyz:
        TransCenterXyz: {}
        TransZRotation: {}
        TransScaling: {}
        TransGaussianNoise:
          mean: 0
          std: 0.002
      rgb:
        TransGaussianNoise:
          mean: 0
          std: 0.02
        TransFeatureDropout:
          dropout_prob: 0.1
      wfm:
        TransSignalScaling: {}
      normals:
        TransGaussianNoise:
          mean: 0
          std: 0.05
      incAngles:
        TransGaussianNoise:
          mean: 0
          std: 0.05
        TransStandardize:
          mean: 2.07
          std: 0.38
      distanceFromScanner:
        TransGaussianNoise

In [15]:
import pandas as pd
from plyfile import PlyData, PlyElement
from torch.nn.functional import softmax
import pickle

report = {} # project, label_level, overall, classwise

for test_project in cfg.data._testProjects_:
    test_ds = FwfDataset(cfg, cfg.data.preprocessing._transformsValidation_, [test_project],
                         return_resiIdx=True, return_projIdx=True)
    proj_name = test_ds.projects[0]['proj_name']
    print(proj_name)
    
    test_ds.subsample_grid(args_test_grid_size, save_inv=True)
    test_ds.compute_neibors_knn(k=cfg.data.num_neib_normalsComputation)
    test_ds.compute_normals_knn()
    test_ds.compute_incAngles()
    test_ds.compute_neibors_knn(k=cfg.data.num_neib_featureExtraction, verbose=True)
    model = FGFeatNetwork(cfg=cfg,
        num_input_feats = test_ds[0]['features_neibors'].shape[-1],
        ).to(device=cfg.general.device)
    model.load_state_dict(torch.load(os.path.join(args_exp_dir,args_model_version), weights_only=True))
    
    test_dl = DataLoader(test_ds, batch_size=cfg.testing.batch_size, num_workers=cfg.general.num_workers)

    

    # initiate containers
    all_preds = [dict() for p in test_ds.projects]

    if args_return_full_probs:
        all_probs = [dict() for p in test_ds.projects]


    for pi, proj in enumerate(test_ds.projects):
        for label_name in test_ds.label_names:

            all_preds[pi][label_name] = torch.ones(size = (test_ds.projects[pi]['xyz_sub'].shape[0],)).to(
                device=cfg.general.device,dtype=torch.int64)*-1
            if args_return_full_probs:
                all_probs[pi][label_name] = torch.ones(size = (test_ds.projects[pi]['xyz_sub'].shape[0],
                    len(cfg.data.label_schema[label_name]))).to(device=cfg.general.device,dtype=torch.float32)*-1

    if args_return_embedding:                
        embeddings = list()
        for pi, proj in enumerate(test_ds.projects):
            embeddings.append(torch.zeros(size=(test_ds.projects[pi]['xyz_sub'].shape[0],cfg.model.embedding_size)).to(device=cfg.general.device))

    # run test loop
    with torch.no_grad():
        for batch in tqdm(test_dl):
            # put batch on device
            for k, v in batch.items():
                batch[k] = v.to(device=cfg.general.device)
            if args_return_embedding:
                out = model(batch, return_embedding=True)
            else:
                out = model(batch)
            probs = dict()
            for k,v in [(k,v) for k,v in out.items() if k != 'embedding']:
                probs[k] = softmax(v,dim=-1)
                preds = torch.argmax(probs[k], dim=-1)
                all_preds[0][k][batch['residual_idx']] = preds # assume dataset has one project inside !

                all_probs[0][k][batch['residual_idx']] = probs[k]
                if args_return_embedding:
                    embeddings[0][batch['residual_idx']] = out['embedding']
        # convert to numpy and upsample
        for k in cfg.data.label_names:
            all_preds[0][k] = all_preds[0][k].cpu().detach().numpy()
            all_preds[0][k] = all_preds[0][k][test_ds.projects[0]['sub_inv']]

            all_probs[0][k] = all_probs[0][k].cpu().detach().numpy()
            all_probs[0][k] = all_probs[0][k][test_ds.projects[0]['sub_inv']]

    

    # map dataset field names to single columns
    column_names_from_fields = {
        'xyz' : ['x','y','z'], 
        'rgb' : ['Red', 'Green','Blue'], 
        'riegl_feats' : ['riegl_reflectance','riegl_amplitude', 'riegl_deviation', 'riegl_targetIndex','riegl_targetCount'], 
        'geom_feats' : ['linearity','planarity','sphericity','omnivariance','anisotropy','eigenentropy',], 
        'normals' : ['nx','ny','nz'], 
        'incAngles' : ['incAngles'], 
        'distanceFromScanner' : ['distanceFromScanner'], 
    }
    # undo normalization
    test_ds.projects[0]['rgb'] *= np.array([[0.21558372, 0.23351644, 0.21213871]])
    test_ds.projects[0]['rgb'] += np.array([[0.29689665, 0.3428666,  0.190237]])
    test_ds.projects[0]['riegl_feats'] *= np.array([[  2.32590898, 2.98518547, 929.71399545, 1., 0.22651793]])
    test_ds.projects[0]['riegl_feats'] += np.array([[-6.90191949, 25.16398933, 26.45952891,  1., 1.03636612]])
    out_data = [test_ds.projects[0][k] for k in cfg.data.scalar_input_fields]
    out_column_names = [s for ss in [column_names_from_fields[f] for f in cfg.data.scalar_input_fields] for s in ss]
    
    # get ground_truth labels
    gt_names = [f"gt_{g}" for g in test_ds.label_names]
    gt_labels = test_ds.projects[0]['labels']
    out_data.append(gt_labels)
    out_column_names += gt_names
    
    # predictions
    pr_names = [f"pr_{g}" for g in test_ds.label_names]
    pr_labels = [all_preds[0][g][:,None] for g in test_ds.label_names]
    out_data += pr_labels
    out_column_names += pr_names

    # probabilities
    if args_return_full_probs:
        prob_names = [i for ii in [cfg.data.label_schema[level] for level in test_ds.label_names] for i in ii] # FIXME: Only works for single levels at a time
        prob_data = [all_probs[0][g] for g in test_ds.label_names]
        out_data += prob_data
        out_column_names += prob_names


    # break
    pcd = pd.DataFrame(np.concatenate(out_data,axis=-1), columns=out_column_names)
    os.makedirs(output_dir, exist_ok=True)
    PlyData([PlyElement.describe(pcd.to_records(index=False),'vertex')]).write(os.path.join(output_dir,proj_name.replace("::","--")+".ply"))

    # save embeddings + inverse indices
    # TODO:
    
    
    # metrics
    # FIXME: shapes, indices, etc. will break for more projects and or more than one label levels
    report[proj_name] = dict()
    for li, l in enumerate(test_ds.label_names):
        metrics = simple_metrics(np.squeeze(gt_labels), np.squeeze(pr_labels), len(cfg.data.label_schema[l]))
        report[proj_name][l] = metrics
    
    # overwrite report
    with open(os.path.join(output_dir,'report.pkl'),'wb') as f:
        pickle.dump(report,f)





         
        

    


Loading '2023-08-28_FW_EingangBauing.FwfProj'; Bounding box IDs = default
2023-08-28_FW_EingangBauing.FwfProj::defaultBbox
Computing neibors for '2023-08-28_FW_EingangBauing.FwfProj::defaultBbox' @ k=20
Computing normals and geom_feats for '2023-08-28_FW_EingangBauing.FwfProj::defaultBbox' @ k=20
Computing incidence angles for '2023-08-28_FW_EingangBauing.FwfProj::defaultBbox'
Computing neibors for '2023-08-28_FW_EingangBauing.FwfProj::defaultBbox' @ k=128


100%|██████████| 23/23 [00:03<00:00,  7.38it/s]


Loading '2024-04-05_FW_Westbahnhof_01.FwfProj'; Bounding box IDs = default
2024-04-05_FW_Westbahnhof_01.FwfProj::defaultBbox
Computing neibors for '2024-04-05_FW_Westbahnhof_01.FwfProj::defaultBbox' @ k=20
Computing normals and geom_feats for '2024-04-05_FW_Westbahnhof_01.FwfProj::defaultBbox' @ k=20
Computing incidence angles for '2024-04-05_FW_Westbahnhof_01.FwfProj::defaultBbox'
Computing neibors for '2024-04-05_FW_Westbahnhof_01.FwfProj::defaultBbox' @ k=128


100%|██████████| 1/1 [00:00<00:00,  2.12it/s]


Loading '2024-07-31_FW_Bruecke_Koenigstr.FwfProj'; Bounding box IDs = [1]
2024-07-31_FW_Bruecke_Koenigstr.FwfProj::bboxId=001
Computing neibors for '2024-07-31_FW_Bruecke_Koenigstr.FwfProj::bboxId=001' @ k=20
Computing normals and geom_feats for '2024-07-31_FW_Bruecke_Koenigstr.FwfProj::bboxId=001' @ k=20
Computing incidence angles for '2024-07-31_FW_Bruecke_Koenigstr.FwfProj::bboxId=001'
Computing neibors for '2024-07-31_FW_Bruecke_Koenigstr.FwfProj::bboxId=001' @ k=128


100%|██████████| 4/4 [00:01<00:00,  3.83it/s]


Loading '2024-08-02_FW_Bruecke_Kasinostrasse.FwfProj'; Bounding box IDs = [0]
2024-08-02_FW_Bruecke_Kasinostrasse.FwfProj::bboxId=000
Computing neibors for '2024-08-02_FW_Bruecke_Kasinostrasse.FwfProj::bboxId=000' @ k=20
Computing normals and geom_feats for '2024-08-02_FW_Bruecke_Kasinostrasse.FwfProj::bboxId=000' @ k=20
Computing incidence angles for '2024-08-02_FW_Bruecke_Kasinostrasse.FwfProj::bboxId=000'
Computing neibors for '2024-08-02_FW_Bruecke_Kasinostrasse.FwfProj::bboxId=000' @ k=128


100%|██████████| 4/4 [00:01<00:00,  3.74it/s]


Loading '2024-07-31_FW_Bruecke_Turmstr.FwfProj'; Bounding box IDs = default
2024-07-31_FW_Bruecke_Turmstr.FwfProj::defaultBbox
Computing neibors for '2024-07-31_FW_Bruecke_Turmstr.FwfProj::defaultBbox' @ k=20
Computing normals and geom_feats for '2024-07-31_FW_Bruecke_Turmstr.FwfProj::defaultBbox' @ k=20
Computing incidence angles for '2024-07-31_FW_Bruecke_Turmstr.FwfProj::defaultBbox'
Computing neibors for '2024-07-31_FW_Bruecke_Turmstr.FwfProj::defaultBbox' @ k=128


100%|██████████| 9/9 [00:01<00:00,  5.85it/s]


In [42]:
# # probs = pd.DataFrame(all_probs[0]['labels_3'], columns=cfg.data.label_schema['labels_3'].keys())



# xyz = pd.DataFrame(np.concatenate([test_ds.projects[0]['xyz'], all_preds[0]['labels_3'][:,None]], axis=1), columns = ['x','y','z','preds_3'])
# pcd = pd.concat([xyz,probs], axis=1)
# os.makedirs(output_dir, exist_ok=True)
# PlyData([PlyElement.describe(pcd.to_records(index=False),'vertex')]).write(os.path.join(output_dir,proj_name.replace("::","--")))

# gt = test_ds.projects[0]['labels'][:,0]
# pr = pcd['preds_3'].to_numpy().astype(np.uint8)
# num_classes = len(cfg.data.label_schema['labels_3'])


36

In [67]:
np.squeeze(pr_labels).shape

(10117061,)

In [68]:
np.squeeze(gt_labels).shape

(10117061,)

In [11]:
metrics

{'iou': array([0.        , 0.21024685, 0.        ,        nan, 0.13517384,
               nan, 0.01812636, 0.04055164, 0.        ,        nan,
        0.        , 0.82774415]),
 'pc_acc': array([0.        , 0.26621843, 0.        ,        nan, 0.15630175,
               nan, 0.01846099, 0.04226558, 0.        , 0.        ,
        0.        , 4.80531818]),
 'miou': 0.1368714272153691,
 'macc': 0.5288564938908277}

In [52]:
len(cfg.data.label_schema[test_ds.label_names]))

KeyValidationError: Incompatible key type 'ListConfig'
    full_key: 
    object_type=dict

In [50]:
gt_labels

array([[11],
       [11],
       [11],
       ...,
       [11],
       [11],
       [11]], dtype=uint8)

In [43]:
out_column_names

['x',
 'y',
 'z',
 'Red',
 'Green',
 'Blue',
 'riegl_reflectance',
 'riegl_amplitude',
 'riegl_deviation',
 'riegl_targetIndex',
 'riegl_targetCount',
 'linearity',
 'planarity',
 'sphericity',
 'omnivariance',
 'anisotropy',
 'eigenentropy',
 'nx',
 'ny',
 'nz',
 'incAngles',
 'distanceFromScanner',
 'gt_labels_3',
 ['pr_labels_3'],
 '_unspecified',
 'asphalt',
 'brick',
 'cable',
 'concrete',
 'marking',
 'mesh',
 'metal',
 'naturalStone',
 'poster',
 'treeTrunk',
 'vegetation']

In [39]:
all_probs[0]['labels_3'].shape

(10117061, 12)

In [36]:
prob_names = [i for ii in [cfg.data.label_schema[level] for level in test_ds.label_names] for i in ii] # FIXME: Only works for single levels at a time
len(prob_names)

12

In [11]:
all_preds[0]['labels_3']

(10117061,)

In [41]:
np.concatenate(out_data,axis=-1).shape

(10117061, 36)

In [15]:
probs_names = test_ds.label_names
probs_names

['labels_3']

In [22]:
list(zip(test_ds.label_names,[cfg.data.label_schema[level] for level in test_ds.label_names]))

[('labels_3',
  {'_unspecified': 0, 'asphalt': 1, 'brick': 2, 'cable': 3, 'concrete': 4, 'marking': 5, 'mesh': 6, 'metal': 7, 'naturalStone': 8, 'poster': 9, 'treeTrunk': 10, 'vegetation': 11})]

['_unspecified',
 'asphalt',
 'brick',
 'cable',
 'concrete',
 'marking',
 'mesh',
 'metal',
 'naturalStone',
 'poster',
 'treeTrunk',
 'vegetation']

In [13]:
all_probs

[{'labels_3': array([[1.5066069e-04, 5.9657796e-06, 6.2773113e-05, ..., 5.5119137e-05,
          2.1044368e-02, 9.7328061e-01],
         [1.5066069e-04, 5.9657796e-06, 6.2773113e-05, ..., 5.5119137e-05,
          2.1044368e-02, 9.7328061e-01],
         [1.5066069e-04, 5.9657796e-06, 6.2773113e-05, ..., 5.5119137e-05,
          2.1044368e-02, 9.7328061e-01],
         ...,
         [4.5500012e-04, 6.4572873e-07, 1.8578959e-05, ..., 2.0549349e-05,
          1.2705003e-02, 9.8146421e-01],
         [8.5954693e-05, 3.4101493e-08, 1.4906019e-05, ..., 1.4475163e-08,
          7.1103009e-03, 9.9056834e-01],
         [4.5500012e-04, 6.4572873e-07, 1.8578959e-05, ..., 2.0549349e-05,
          1.2705003e-02, 9.8146421e-01]], dtype=float32)}]

In [6]:
# save embeddings together with inverse indices
embeddings
test_ds.projects[0]['sub_inv']

NameError: name 'embeddings' is not defined

In [9]:
test_ds.projects[0]['xyz'].shape

(10117061, 3)

In [7]:
gt_labels.shape

(10117061, 1)

In [5]:
out_column_names

['x',
 'y',
 'z',
 'Red',
 'Green',
 'Blue',
 'riegl_reflectance',
 'riegl_amplitude',
 'riegl_deviation',
 'riegl_targetIndex',
 'riegl_targetCount',
 'linearity',
 'planarity',
 'sphericity',
 'omnivariance',
 'anisotropy',
 'eigenentropy',
 'nx',
 'ny',
 'nz',
 'incAngles',
 'distanceFromScanner',
 'gt_labels_3']

In [ ]:
out_data.shape

In [ ]:
test_ds.projects[0]['labels']

In [ ]:
test_ds.projects[0]['labels']

gt_names

In [94]:
# iou_c = tp_c / (fp_c + fn_c + tp_c)
# pc_acc = tp_c / ()





In [72]:
iou[~valid_classes] = np.nan

In [ ]:
miou

In [ ]:
np.nanmean(iou)

In [ ]:
macc

In [ ]:
iou

In [ ]:
pc_acc.mean()

In [ ]:
iou.mean()

In [ ]:
gt.shape

In [ ]:
test_ds.projects[0]['proj_name']

In [ ]:
np.mean((gt == pr))

In [ ]:
# from torch.nn.functional import softmax
# # run test loop
# with torch.no_grad():
#     for batch in tqdm(test_dl):
#         # put batch on device
#         for k, v in batch.items():
#             batch[k] = v.to(device=cfg.general.device)
#         out = model(batch)
#         probs = dict()
#         for k,v in out.items():
#             probs[k] = softmax(v,dim=-1)
#             preds = torch.argmax(probs[k], dim=-1)
#             all_preds_sub[0][k][batch['residual_idx']] = preds # assume dataset has one project inside !

#             all_probs_sub[0][k][batch['residual_idx']] = probs[k]
#     # convert to numpy and upsample
#     for k in cfg.data.label_names:
#         all_preds_sub[0][k] = all_preds_sub[0][k].cpu().detach().numpy()
#         all_preds_sub[0][k] = all_preds_sub[0][k][test_ds.projects[0]['sub_inv']]

#         all_probs_sub[0][k] = all_probs_sub[0][k].cpu().detach().numpy()
#         all_probs_sub[0][k] = all_probs_sub[0][k][test_ds.projects[0]['sub_inv']]



            
        


In [ ]:
all_preds[0]['labels_3'][None,:].shape

In [ ]:
cfg.data.label_schema['labels_3']
all_probs

In [8]:
# import pandas as pd
# from plyfile import PlyData, PlyElement

# probs = pd.DataFrame(all_probs_sub[0]['labels_3'], columns=cfg.data.label_schema['labels_3'].keys())

# xyz = pd.DataFrame(np.concatenate([test_ds.projects[0]['xyz'], all_preds_sub[0]['labels_3'][:,None]], axis=1), columns = ['x','y','z','preds_3'])
# pcd = pd.concat([xyz,probs], axis=1)
# PlyData([PlyElement.describe(pcd.to_records(index=False),'vertex')]).write(f"./_temp/test_preds_with_probs.ply")


In [ ]:
np.unique(aps, return_counts=True)

In [ ]:
probs[k]

In [ ]:
(test_ds.projects[0]['labels_sub'].reshape(-1) == aps).astype(np.float32).mean()
aps[test_ds][0]['inv_inds']

In [ ]:
len(cfg.data.label_schema[label_name])

In [ ]:
label_name

In [ ]:
test_ds.projects[0]['sub_inv'].shape